> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 4.2 - APIs


## Exercícios 🔭🌌🪐

Vamos acessar as APIs da NASA para ver algumas imagens interessantes capturadas universo afora!

#### Q1.
Crie uma chave no site oficial:
* https://api.nasa.gov

Vamos armazenar a chave de forma segura! <br>
Salve a sua chave em um arquivo `key.json` na forma:
`API_KEY=SUA_CHAVE`

Adicione o nome do arquivo `key.json` ao `.gitignore` do repositório que você fará upload da atividade.
Para isso basta abrir o arquivo `.gitignore` na pasta raíz do repositório (ou criar um caso ele não exista). Dentro do arquivo, apenas adiciona o nome do arquivo que deseja ignorar.

#### Q2. 🛰 Astronomy Picture of the Day (APOD) 🌌
> Antes de fazer os exercícios, devo te lembrar que existem limites de acesso às APIs, descritas na página principal, portanto pega leve na tentativa e erro na hora de testar seu código.

<img width=500 src=https://apod.nasa.gov/apod/image/2407/M24-HaLRGB-RC51_1024.jpg>

A primeira API que acessaremos é a mais popular de todas: astronomy picture of the day (foto astronômica do dia).

Faça uma requisição GET para a URL da API que retorna a imagem do dia! Essa é fácil já que são os valores padrão da rota principal:
* URL base: `'https://api.nasa.gov/planetary/apod'`
* Endpoint: não precisa preencher, acessaremos a raíz da API.
* Query params: preencha `api_key` com a sua chave de autenticação. Se animar mexer em outros parâmetros veja [a documentação](https://api.nasa.gov).

Ao receber a resposta (um json), você deve:
* Imprimir os campos `copyright` e `explanation`
* Com as biblioteca scikit-images e matplotlib, apresente a imagem a partir do campo `url` ou `hdurl`, e preencha o título do plot com o campo `title` do json. Uma dica de código a seguir.
```python
from skimage import io
img = io.imread(url)
## plot a matriz img com matplotlib (imshow)
```   

In [ ]:
import requests
from skimage import io
import matplotlib.pyplot as plt

# Use sua própria chave de API da NASA aqui.
# Você pode usar "DEMO_KEY" para testes rápidos, mas o limite é bem baixo.
API_KEY = "DEMO_KEY"  # substitua por sua chave pessoal, se tiver

base_url = "https://api.nasa.gov/planetary/apod"

params = {
    "api_key": API_KEY,
}

resp_apod = requests.get(base_url, params=params)
resp_apod.raise_for_status()  # dispara erro se a requisição falhar

data_apod = resp_apod.json()

print("==== Astronomy Picture of the Day ====")
print("Título:", data_apod.get("title", ""))
print()

print("Copyright:", data_apod.get("copyright", "N/A"))
print()
print("Explanation:")
print(data_apod.get("explanation", ""))

# Usa a URL da imagem (hdurl se existir, senão url normal)
image_url = data_apod.get("hdurl") or data_apod.get("url")

if image_url:
    img = io.imread(image_url)

    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.axis("off")
    plt.title(data_apod.get("title", "Astronomy Picture of the Day"))
    plt.show()
else:
    print("\nNenhuma URL de imagem encontrada na resposta da API.")

#### Q3. Limites
A partir da resposta da query anterios, imprima o header da resposta e consulte os atributos:
* X-RateLimit-Limit: o limite total de requisições da sua chave de API
* X-RateLimit-Remaining: o limite restante de requisições da sua chave de API

In [ ]:
# Q3) Limites de uso da API
# Usamos o objeto `resp_apod` obtido na célula anterior.

print("==== Headers da resposta APOD ====")
for k, v in resp_apod.headers.items():
    print(f"{k}: {v}")

limit_total = resp_apod.headers.get("X-RateLimit-Limit")
limit_restante = resp_apod.headers.get("X-RateLimit-Remaining")

print("\nX-RateLimit-Limit   :", limit_total)
print("X-RateLimit-Remaining:", limit_restante)

### Q4. Mars Rover Photos 🚀🚙 📷

<img width=500 src=https://www.nasa.gov/wp-content/uploads/2019/10/pia23378-16.jpg>

Essa API retorna dados (incluindo imagens capturadas) sobre os veículos que hoje habitam o planeta Marte. São os rovers `opportunity`, `spirit` e o mais famoso, o `curiosity` (da foto acima).

Antes de requisitar imagens, vamos ver o relatório de dados coletados por um deles, o `curiosity`. Isso vai nos ajudar a montar a query de imagens coletadas.

Faça uma requisição GET para a seguinte URL:
* URL base: `'https://api.nasa.gov/mars-photos/api/v1'`
* endpoint: `'/manifests/{nome_do_rover}'`
* query parameters: preencha `api_key` com a sua chave de autenticação.

Extraia o json da resposta retornada. O campo principal é o `'photo_manifest'`, do qual queremos acessar os seguintes valores:
* `max_sol`: Máximo "dia marciano" de coleta de fotos. O dia marciano tem 24 horas, 39 minutos e 35 segundos.
* `max_date`: Última data terrestre de coleta de fotos, na forma `'aaaa-mm-dd'`.

Imprima esses dois atributos da resposta e os use no próximo exercício para coletar as fotos mais recentes tiradas. 

In [ ]:
import requests

# Vamos reaproveitar a mesma API_KEY definida na Q2
base_url_mars = "https://api.nasa.gov/mars-photos/api/v1"
rover_name = "curiosity"  # você pode trocar para "opportunity" ou "spirit"

manifest_url = f"{base_url_mars}/manifests/{rover_name}"

params = {
    "api_key": API_KEY,
}

resp_manifest = requests.get(manifest_url, params=params)
resp_manifest.raise_for_status()

data_manifest = resp_manifest.json()
photo_manifest = data_manifest["photo_manifest"]

max_sol = photo_manifest["max_sol"]
max_date = photo_manifest["max_date"]

print("==== Manifesto de fotos do rover ====")
print("Rover      :", photo_manifest["name"])
print("Max sol    :", max_sol)
print("Max date   :", max_date)
print("Total fotos:", photo_manifest["total_photos"])

#### Q5.

Faça uma requisição GET para a URL da API que retorna links para as imagens coletadas pelos rovers.

* URL base: `'https://api.nasa.gov/mars-photos/api/v1'`
* Endpoint: `/rovers/{nome_do_rover}/photos`
* Query params sugeridos: 
    * `api_key`: sua chave de autenticação.
    * `sol`: dia marciano que deseja coletar (de 0 a `max_sol` coletado anteriormente)
    * `page`: você pode paginar entre as respostas! São retornados 25 resultados por página.

A resposta esperada estará no formato a seguir, uma lista no campo `'photos'` onde cada item é um dicionário com os dados da foto tirada. Dentre os dados há o campo `camera` indicando qual das câmeras do rover tirou a foto. As fotos mais interessantes (na minha opinião, claro) são das câmeras de navegação (`"name": "NAVCAM"`) e as de prevenção de colisão (frente: `"name": "FHAZ"` e trás `"name": "RHAZ"`) onde dá pra ver partes do robô!

**Seu trabalho é**:
* Paginar a requisição acima até que a resposta seja `None`
* Escolher uma ou mais câmeras (ex: `NAVCAM`, `FHAZ`, `RHAZ`), e em um laço de repetição plotar todas as imagens retornadas daquela câmera. Use novamente as bibliotecas scikit-image e matplotlib. 
  * O título da imagem deve ter a página da requisição, nome da câmera e id da imagem.

```json
{
  "photos": [
    {
      "id": 1228212,
      "sol": 4102,
      "camera": {
        "id": 20,
        "name": "FHAZ",
        "rover_id": 5,
        "full_name": "Front Hazard Avoidance Camera"
      },
      "img_src": "https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/04102/opgs/edr/fcam/FLB_761645828EDR_F1060660FHAZ00302M_.JPG",
      "earth_date": "2024-02-19",
      "rover": {
        "id": 5,
        "name": "Curiosity",
        ...
      }
    }
    {
      "id": 1228213,
      "sol": 4102, 
      ...
    }
```



In [ ]:
from skimage import io
import matplotlib.pyplot as plt
import math
import requests

# Usamos:
# - base_url_mars definido na Q4
# - API_KEY definido na Q2
# - max_sol obtido na Q4

rover_name = "curiosity"
photos_url = f"{base_url_mars}/rovers/{rover_name}/photos"

# Escolha das câmeras a exibir (você pode alterar)
cameras_interesse = {"FHAZ", "RHAZ", "NAVCAM"}

sol = max_sol  # pega o último sol disponível do manifesto
page = 1
todas_fotos = []

print(f"Coletando fotos do rover {rover_name} no sol {sol}...")

while True:
    params = {
        "api_key": API_KEY,
        "sol": sol,
        "page": page,
    }

    resp_photos = requests.get(photos_url, params=params)
    resp_photos.raise_for_status()

    data_photos = resp_photos.json()
    photos = data_photos.get("photos", [])

    if not photos:
        # Quando a página não retorna mais fotos, encerramos a paginação
        break

    todas_fotos.extend(photos)
    print(f"Página {page}: {len(photos)} fotos")
    page += 1

print(f"Total de fotos retornadas para o sol {sol}: {len(todas_fotos)}")

# Filtra pelas câmeras de interesse
fotos_filtradas = [
    p for p in todas_fotos
    if p["camera"]["name"] in cameras_interesse
]

print(f"Total de fotos para as câmeras {cameras_interesse}: {len(fotos_filtradas)}")

# Vamos exibir até N imagens em um grid
N = min(6, len(fotos_filtradas))

if N == 0:
    print("Nenhuma foto encontrada para as câmeras selecionadas.")
else:
    cols = 3
    rows = math.ceil(N / cols)

    plt.figure(figsize=(4 * cols, 4 * rows))

    for i, photo in enumerate(fotos_filtradas[:N], start=1):
        img_url = photo["img_src"]
        img = io.imread(img_url)

        ax = plt.subplot(rows, cols, i)
        ax.imshow(img)
        ax.axis("off")

        titulo = (
            f"{photo['rover']['name']} - {photo['camera']['name']}\n"
            f"Sol {photo['sol']} ({photo['earth_date']})"
        )
        ax.set_title(titulo, fontsize=8)

    plt.tight_layout()
    plt.show()